In [3]:
from keras.applications import inception_v3
from keras import backend as K
import numpy as np
import scipy
import matplotlib.pyplot as plt
from keras.preprocessing import image
from utils import *
%matplotlib inline

Using TensorFlow backend.
/home/fabrice/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


# Deep Dream in Theory

Deep Dream is a computer vision algorithm developed at google in 2014.
It modify an image with some dream-like shapes.


<img src="image/deep-dream.jpg" style="width:700px;height:400px;">

The code of this jupyter-notebook comes from Francois Chollet book "Deep Learning with Python",
it is also available on the github of the book: https://github.com/fchollet/deep-learning-with-python-notebooks

Francois Chollet is also the author of the library we will use here Keras.

<img src="image/keras-book.png" style="width:300px;height:450px;">

DeepDream is a Generative model, as oppose to the classical predictive models.
Generative models generate "something new" on their own. 

Predictive models:
- Classification
- Regression

Generative models:
- GAN: generative adversial network
- VAE: variational auto encoder
- Language modeling
- Style Transfert
- Deep Dream

For our Implementation we will use an existing Neural Network, called Inception.

Inception was trained on imagenet, a famous dataset of picture labeled with classes. 1000 classes exist in the dataset: cat, elephant, phone, house... but it is mostly cats and dogs...

Using a trained network is called <b>transfert learning</b> because we take advantage of a NN already trained to perform a different task.

<img src="image/inception.png" style="width:800px;height:350px;">

Below is a simplification the Neural Network architecture:

<img src="image/neural-network.png">

Now we going to hack this NN to achieve our goal: Modify the input picture!

<img src="image/deep-dream-network.png">

Gradient descent: w = w - lr * grad_w

Gradient ascent: w = w + lr * grad_w

Each layer of a NN contains fitlers.

Each filter capture a shape, a texture, a color, of an image.

Early layers in a NN capture simple shapes: vertical edges, simple texture...

Later layers in a NN capture more complex shapes: ears, faces, fish...

<img src="image/layers-observation.png">

# Deep Dream in Practice

## Load the inception model

In [4]:
K.set_learning_phase(0) # disable all training of the model weights
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

## Choose layers to compute the loss


In [4]:
# four layer will be involve in calculation of the loss
# each layer will have a specific weight in the loss.
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
    }

In [5]:
# LOSS we will define the loss as a weighted sum of L2 norm of all filters in several layers
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.) # initialize loss to 0
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # number of activation in a layer
    # avoid border effect by selecting 2:-2 
    loss += coeff * K.sum(K.square(activation[:,2:-2, 2:-2, :])) / scaling

## Define our gradient ascent

In [6]:
# Gradient ascent
dream = model.input
grads = K.gradients(loss, dream)[0]  # gradient of the input with regard to the loss.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalize gradient (like clipping)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
   
def gradient_ascent(picture, iterations, learning_rate, max_loss=None):
    for i in range(iterations):
        loss_value, gradient_values = eval_loss_and_grads(picture)
        if (max_loss!=None) and (loss_value>max_loss):
            # Stop the algorithm after a Maximum loss threshold 
            break
        print('...Loss value at', i, ':', loss_value)
        picture = picture + learning_rate*gradient_values
    return picture

## Generate the new images

In [14]:
# running gradient ascent of different sceles of image
def main(base_image_path):
    learning_rate = 0.01
    num_octave = 3
    octave_scale = 1.4
    iterations = 20
    max_loss = 10 

    img = preprocess_image(base_image_path)
    original_shape = img.shape[1:3]
    successive_shapes = [original_shape]
    for i in range(1, num_octave):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)
    successive_shapes = successive_shapes[::-1]
    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0]) # srunk to first scale

    for shape in successive_shapes:
        print('Processing image shape', shape)
        img = resize_img(img, shape)
        img = gradient_ascent(img,
                              iterations=iterations,
                              learning_rate=learning_rate,
                              max_loss=max_loss)
        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
        same_size_original = resize_img(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail
        shrunk_original_img = resize_img(original_img, shape)
        save_img(img, fname='output/dream_at_scale_' + str(shape) + '.png')
    save_img(img, fname='output/final_dream.png')

In [10]:
# Run the generative model
main('./dream/sky.jpg')

Processing image shape (85, 153)
...Loss value at 0 : 0.03275671
...Loss value at 1 : 0.04306922
...Loss value at 2 : 0.05659038
...Loss value at 3 : 0.06841916
...Loss value at 4 : 0.080613665
...Loss value at 5 : 0.093742795
...Loss value at 6 : 0.10601663
...Loss value at 7 : 0.11790989
...Loss value at 8 : 0.12908259
...Loss value at 9 : 0.13971855
...Loss value at 10 : 0.1486108
...Loss value at 11 : 0.15830845
...Loss value at 12 : 0.167124
...Loss value at 13 : 0.17638376
...Loss value at 14 : 0.18385811
...Loss value at 15 : 0.19145077
...Loss value at 16 : 0.19700456
...Loss value at 17 : 0.20261818
...Loss value at 18 : 0.21097225
...Loss value at 19 : 0.21771075
Processing image shape (120, 214)
...Loss value at 0 : 0.7252682
...Loss value at 1 : 1.0682368
...Loss value at 2 : 1.4923152
...Loss value at 3 : 1.8387153
...Loss value at 4 : 2.1870494
...Loss value at 5 : 2.4675007
...Loss value at 6 : 2.7788463
...Loss value at 7 : 3.0096638
...Loss value at 8 : 3.2916827
...Lo

# Exercices

<span style=color:blue>Pictures of homogenous view (sky, grass land, desert) are actually easier for the neural network to modify, has there is not much petterns already in place<span>

## Playground exercice one:
The layers of the neural network contain filters that are the patterns that will appear on your picture.

Early layers contain basic patterns, like horizontal lines, simple textures... 
Later layers contain more elaborate patterns, like house shape, eyes, ears...

<b>Modify the layers used in</b> `layer_contibutions` <b>for computing the loss, and observe the impact it has on your generated picture.</b>

The modification can happen in three ways:

1. Modify the number of layers.
2. Modify the weight of the layers
3. Change which layers are used.

Hint: 
- Look at the `model.summary()` to know the name of the layers.
- The layers named `mixed` and `conV2D` are the most significant to use.

<span style=color:red>After modifying `layer_contribution` you need to run the other cells to update the computational graph<span>


In [1]:
##############################
# Edit the contibuting layers

layer_contributions = {
    # enter layers here
    }

In [ ]:
####################################
# RUN THIS CELL TO UPDATE THE GRAPH
# NO NEED TO CHANGE THIS CODE


layer_dict = dict([(layer.name, layer) for layer in model.layers]) # map layer_name -> instance

loss = K.variable(0.) # initialize loss to 0
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # number of activation in a layer
    # avoid border effect by selecting 2:-2 
    loss += coeff * K.sum(K.square(activation[:,2:-2, 2:-2, :])) / scaling
    # Gradient ascent
    
dream = model.input
grads = K.gradients(loss, dream)[0]  # gradient of the input with regard to the loss.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalize gradient (like clipping)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
   
def gradient_ascent(picture, iterations, learning_rate, max_loss=None):
    for i in range(iterations):
        loss_value, gradient_values = eval_loss_and_grads(picture)
        if (max_loss!=None) and (loss_value>max_loss):
            # Stop the algorithm after a Maximum loss threshold 
            break
        print('...Loss value at', i, ':', loss_value)
        picture = picture + learning_rate*gradient_values
    return picture

In [ ]:
# Run the generative model
image_path = # enter your image here
main(image_path)

## Playground exercice two:
The learning rate define the speed at which the NN will reach a local optimum.

The max_loss cap the algorithm to not exceed a certain loss.

<b>Modify the max_loss to see a more dramatic change in the image.</b>

<b>Modify the learning rate to observe a speed-up in the loss increase.</b>


Hint: 
- This will happens in the `main()` function.
- A modification or `learning_rate` by a factor 10 is already a big change.

<span style=color:red>If you only modify the `main()` function you won't need to update the computational graph<span>

In [68]:
##############################
# Edit the contibuting layers

layer_contributions = {
    # enter layers here
    }

In [69]:
####################################
# RUN THIS CELL TO UPDATE THE GRAPH
# NO NEED TO CHANGE THIS CODE


layer_dict = dict([(layer.name, layer) for layer in model.layers]) # map layer_name -> instance

loss = K.variable(0.) # initialize loss to 0
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # number of activation in a layer
    # avoid border effect by selecting 2:-2 
    loss += coeff * K.sum(K.square(activation[:,2:-2, 2:-2, :])) / scaling
    # Gradient ascent
    
dream = model.input
grads = K.gradients(loss, dream)[0]  # gradient of the input with regard to the loss.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalize gradient (like clipping)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
   
def gradient_ascent(picture, iterations, learning_rate, max_loss=None):
    for i in range(iterations):
        loss_value, gradient_values = eval_loss_and_grads(picture)
        if (max_loss!=None) and (loss_value>max_loss):
            # Stop the algorithm after a Maximum loss threshold 
            break
        print('...Loss value at', i, ':', loss_value)
        picture = picture + learning_rate*gradient_values
    return picture

In [70]:
############################################################
# Edit the learning rate to speed up the modification
# Edit the max_loss to increase the modification of the image

def main(base_image_path):
    learning_rate = 0.01
    num_octave = 3
    octave_scale = 1.4
    iterations = 20
    max_loss = 50

    img = preprocess_image(base_image_path)
    original_shape = img.shape[1:3]
    successive_shapes = [original_shape]
    for i in range(1, num_octave):
        shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
        successive_shapes.append(shape)
    successive_shapes = successive_shapes[::-1]
    original_img = np.copy(img)
    shrunk_original_img = resize_img(img, successive_shapes[0]) # srunk to first scale

    for shape in successive_shapes:
        print('Processing image shape', shape)
        img = resize_img(img, shape)
        img = gradient_ascent(img,
                              iterations=iterations,
                              learning_rate=learning_rate,
                              max_loss=max_loss)
        upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
        same_size_original = resize_img(original_img, shape)
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail
        shrunk_original_img = resize_img(original_img, shape)
        save_img(img, fname='output/dream_at_scale_' + str(shape) + '.png')
    save_img(img, fname='output/final_dream.png')

In [ ]:
# Run the generative model
image_path = # enter your image here
main(image_path)

## Advance exercice
<b>Observe a single filter pattern</b>.

Layers in a neural network are composed of several filters. Each filter recognize a particular pattern in an image.

To observe a single filter it is quite similar to what we have done until now. First we not gonna imput an image but noise, this will be easier for the network to change it. Then instead of modifying the input based on the activations of
entire layers, we just select the activation from a single filter in this layer. The gradient ascent will create an input that optimize what the filter have learned to recognize.

Steps:
1. Create a noisy input picture with numpy.
2. Select only one filter of a layer in the loss function
3. Input the noise into the gradient ascent instead of an image, and allow max_loss to bi pretty big.

<span style=color:red>Run every cell from creating the noise each time you make a change to a parameter<span>

In [ ]:
##############################################################
# Use Numpy to create a picture with noise in it
# it is a 3 dimensional array, with random numbers around 128.

noise_input = # TODO
#plt.imshow(noise_input[:,:,1])

In [ ]:
##############################
# Edit the contributing layers
layer_contributions = {

    }

In [ ]:
##############################################################
# Modify the LOSS to look only at one filter in a given layer
layer_dict = dict([(layer.name, layer) for layer in model.layers]) # map layer_name -> instance

loss = K.variable(0.) # initialize loss to 0
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32')) # number of activation in a layer
    # avoid border effect by selecting 2:-2 
    loss += coeff * K.sum(K.square(activation[:,2:-2, 2:-2, :])) / scaling

In [ ]:
####################################
# RUN THIS CELL TO UPDATE THE GRAPH
# NO NEED TO CHANGE THIS CODE
dream = model.input
grads = K.gradients(loss, dream)[0]  # gradient of the input with regard to the loss.
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalize gradient (like clipping)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
   
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss!=None and loss_value>max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step*grad_values
    return x

In [ ]:
##############################################################################
# Modify the main function to allow high loss values
# Have the noisy image input for the gradient ascent instead of a load picture

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = # allow big loss value



def preprocess_noise(noise):
    img = np.expand_dims(noise, axis=0)
    img = inception_v3.preprocess_input(img) 
    return img

# replaced preprocess_image() with preprocess_noise()
img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])


for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='output/dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname='output/final_dream.png')